In [1]:
# A notebook to give proof of concept for the cocktail recipe app that is inventory "aware"
# Initial Imports
import pandas as pd
import numpy as np
import os
import openai
from dotenv import load_dotenv

# Load .env enviroment variables
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")


In [2]:
new_df = pd.read_csv("inventory.csv")
new_df.head()

,Name,Quantity,Volume per Unit (ml),Cost per Unit,Cost per ml
0,Vodka,6,750,51.758815,0.069012
1,Gin,2,1000,20.383485,0.020383
2,Rum,1,1000,75.476875,0.075477
3,Tequila,9,750,43.807963,0.058411
4,Whiskey,5,750,90.563148,0.120751


In [3]:
# Drop the "Cost per ml" column
new_df = new_df.drop(columns=["Cost per ml"])

In [5]:
# Add the "Cost per ml" column back in with the correct values
new_df["Cost per ml"] = new_df["Cost per Unit"] / new_df["Volume per Unit (ml)"]

In [7]:
# Now add a "Cost per oz" column by dividing the "Cost per ml" column by 29.5735
new_df["Cost per oz"] = new_df["Cost per ml"] * 29.5735

new_df.head()

,Name,Quantity,Volume per Unit (ml),Cost per Unit,Cost per ml,Cost per oz
0,Vodka,6,750,51.758815,0.069012,2.040919
1,Gin,2,1000,20.383485,0.020383,0.602811
2,Rum,1,1000,75.476875,0.075477,2.232115
3,Tequila,9,750,43.807963,0.058411,1.727406
4,Whiskey,5,750,90.563148,0.120751,3.571026


In [8]:
# Export the new dataframe to a csv file
new_df.to_csv("inventory.csv", index=False)

In [28]:
# Additional imports
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader

# Load the csv file into the CSVLoader
loader = CSVLoader(file_path = "inventory.csv", source_column="Name", csv_args={"delimiter": ","})
data = loader.load()

data[:5]

[Document(page_content='ItemID: 1\nName: Vodka\nType: Spirit\nQuantity: 6\nVolume per Unit (ml): 750\nCost per Unit: 51.758815405142514\nCost per ml: 0.06901175387352335', metadata={'source': 'Vodka', 'row': 0}),
 Document(page_content='ItemID: 2\nName: Gin\nType: Spirit\nQuantity: 2\nVolume per Unit (ml): 1000\nCost per Unit: 20.383484696146947\nCost per ml: 0.020383484696146946', metadata={'source': 'Gin', 'row': 1}),
 Document(page_content='ItemID: 3\nName: Rum\nType: Spirit\nQuantity: 1\nVolume per Unit (ml): 1000\nCost per Unit: 75.47687481439222\nCost per ml: 0.07547687481439222', metadata={'source': 'Rum', 'row': 2}),
 Document(page_content='ItemID: 4\nName: Tequila\nType: Spirit\nQuantity: 9\nVolume per Unit (ml): 750\nCost per Unit: 43.807963027050874\nCost per ml: 0.05841061736940117', metadata={'source': 'Tequila', 'row': 3}),
 Document(page_content='ItemID: 5\nName: Whiskey\nType: Spirit\nQuantity: 5\nVolume per Unit (ml): 750\nCost per Unit: 90.56314762228048\nCost per ml:

In [10]:
# Create the embeddings
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(data, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [34]:
from langchain.agents import create_csv_agent
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools, Tool, tool

In [32]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0.5)
csv_agent = create_csv_agent(llm, "inventory.csv", verbose = True)


In [70]:
from langchain.utilities import PythonREPL
from langchain.chains import LLMMathChain

# Create the Python REPL
python_repl = PythonREPL()

# Create the math chain
math_chain = LLMMathChain(llm = llm, verbose = True)

# Create the tools
tools = [
    Tool.from_function(
        name = "python_repl",
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
        func=python_repl.run
    ),
    Tool.from_function(
        name = "Calculator",
        description="Useful for when you need to answer questions about math.",
        func=math_chain.run
    )
]

In [58]:
tools

[Tool(name='python_repl', description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method PythonREPL.run of PythonREPL(globals={}, locals={})>, coroutine=None),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=True, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the ou

In [71]:
agent = initialize_agent(tools, llm, agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [64]:
liquors = df['Name'].values

In [65]:
liquors

array(['Vodka', 'Gin', 'Rum', 'Tequila', 'Whiskey', 'Brandy', 'Scotch',
       'Bourbon', 'Cognac', 'Absinthe', 'Sake', 'Rye', 'Mezcal',
       'Amaretto', 'Baileys', 'Campari', 'Cointreau', 'Frangelico',
       'Grand Marnier', 'Jägermeister', 'Kahlúa', 'Limoncello', 'Sambuca',
       'Southern Comfort', 'Tia Maria', 'Triple Sec', 'Pisco', 'Port',
       'Sherry', 'Vermouth', 'Aperol', 'Bitters', 'Chartreuse',
       'Chambord', 'Fernet Branca', 'Galliano', 'Goldschlager', 'Malibu',
       'Midori', 'Peach Schnapps', 'Peppermint Schnapps', 'Sloe Gin',
       'St Germain', 'Umeshu', 'Anisette', 'Aquavit', 'Becherovka',
       'Bénédictine', 'Crème de Cacao', 'Crème de Menthe'], dtype=object)

In [72]:
agent.run(f"Can you help me create a cocktail from liquors {liquors} in my inventory?")




> Entering new AgentExecutor chain...
Question: Would you like a cocktail recipe that includes all the liquors in your inventory?
Thought: I will use the Cocktail tool to generate a recipe that includes all of the liquors in the inventory.
Action:
```
{
  "action": "Cocktail",
  "action_input": "all"
}
```

Observation: Cocktail is not a valid tool, try another one.
Thought:I will use the python_repl tool to create a cocktail recipe that includes all of the liquors in the inventory.

> Finished chain.


'I will use the python_repl tool to create a cocktail recipe that includes all of the liquors in the inventory.'



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:

Human: Can you create a good cocktail using some ingredients from my inventory?
Assistant: Sure! I can't really tell you what kind of cocktail to make, as that depends on your personal taste preferences. However, I can suggest that you try making a classic gin and tonic using the gin from your inventory. Alternatively, you could try making an Old Fashioned using the whiskey and bitters in your inventory. Cointreau can be used in many cocktails as a triple sec substitute or in Margaritas. It all depends on your taste.
Human: How about one that uses the liquor that I have the most of in the inventory?
Assistant: I'm sorry, I don't have access to your inventory or information about what cocktail you are interested in making. Please provide me with more details and I'll do my best to help you.
Fol

In [19]:
[print(result["answer"])]

I'm sorry, I am an AI language model and I don't have information about which craft cocktail you can make with Mezcal and another liquor from your inventory as it depends on personal preference and recipe expertise. However, you can search for cocktail recipes online or consult a mixologist for recommendations.


[None]